In [2]:
import numpy as np
import torch
import torch.nn as nn
import torchbnn as bnn  # torchbnn library for BNN layers
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from tqdm import tqdm

In [45]:

class BayesianNN(nn.Module):
    def __init__(self, input_dim, prior_mu, prior_sigma, layer1_units, layer2_units):
        super(BayesianNN, self).__init__()
        # Define prior parameters
        prior_mu = 0.0  # Mean of the prior distribution
        prior_sigma = 0.1  # Standard deviation of the prior distribution
        
        # Initialize Bayesian layers with the specified priors
        self.fc1 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=input_dim, out_features=layer1_units)
        self.fc2 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=layer1_units, out_features=layer2_units)
        self.fc3 = bnn.BayesLinear(prior_mu, prior_sigma, in_features=layer2_units, out_features=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [47]:
def train_bayesian_nn(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)  # Shape: (batch_size, 1)
        
        # Ensure y_batch has the same shape as y_pred
        y_batch = y_batch.unsqueeze(1)  # Convert y_batch shape to (batch_size, 1) if needed
        
        loss = criterion(y_pred, y_batch)  # Now both are of shape (batch_size, 1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


In [81]:
# # Evaluation function
# def evaluate_bayesian_nn(model, loader):
#     model.eval()
#     all_preds = []
#     with torch.no_grad():
#         for X_batch, _ in loader:
#             y_pred = model(X_batch)
#             print("y_pred shape: ", y_pred.shape)
#             if y_pred.dim() > 1:  # Check if y_pred is not a scalar
#                 all_preds.extend(y_pred.round().squeeze().cpu().numpy())
#             else:
#                 all_preds.append(y_pred.round().cpu().numpy())  # For scalar, append directly

#     return np.array(all_preds)
def evaluate_bayesian_nn(model, val_loader):
    model.eval()
    all_preds = []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = model(X_batch)

            # Round predictions and move to CPU for compatibility
            y_pred = y_pred.round().cpu().numpy()

            # If y_pred is a scalar, wrap it in a list, otherwise ensure it's a list-like object
            if np.isscalar(y_pred):
                y_pred = [y_pred]  # Wrap scalar in a list
            elif len(y_pred.shape) == 1:  # If it's already a 1D array, no need to squeeze
                y_pred = y_pred.tolist()  # Convert it to a list directly

            # Extend the list of predictions
            all_preds.extend(y_pred)

    return np.array(all_preds)





In [75]:
# Function to run BNN for each value of n
def run_bayesian_nn(n, hyper_params):
    # Load data
    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')
    
    # Split data into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    # Convert data to PyTorch tensors and create DataLoaders
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32).clone().detach(), torch.tensor(y_train, dtype=torch.float32).clone().detach()
    X_val, y_val = torch.tensor(X_val, dtype=torch.float32).clone().detach(), torch.tensor(y_val, dtype=torch.float32).clone().detach()
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32).clone().detach(), torch.tensor(y_test, dtype=torch.float32).clone().detach()
    
    # Convert data to PyTorch tensors and create DataLoaders
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=hyper_params['batch_size'], shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=hyper_params['batch_size'])
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

    # Initialize the model, loss function, and optimizer
    input_dim = X_train.shape[1]
    model = BayesianNN(input_dim=input_dim, prior_mu=hyper_params['prior_mu'], prior_sigma=hyper_params['prior_sigma'], layer1_units=hyper_params['layer1_units'], layer2_units=hyper_params['layer2_units'])

    
    criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
    optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate'])
    
    # Train the model and track training loss
    training_losses = []
    for epoch in range(hyper_params['epochs']):
        train_loss = train_bayesian_nn(model, train_loader, criterion, optimizer)
        training_losses.append(train_loss)
        print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}")
    
    # Validate the model
    y_val_pred = evaluate_bayesian_nn(model, val_loader)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"Validation Accuracy for n={n}: {val_accuracy:.4f}")
    
    # Test the model
    y_test_pred = evaluate_bayesian_nn(model, test_loader)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy for n={n}: {test_accuracy:.4f}")
    
    return test_accuracy

In [ ]:


space = [
    Real(1e-5, 1e-1, name='learning_rate'),  # Learning rate
    Integer(16, 128, name='batch_size'),     # Batch size
    Integer(32, 128, name='layer1_units'),   # Number of units in layer 1
    Integer(16, 64, name='layer2_units'),    # Number of units in layer 2
    Real(0.0, 1.0, name='prior_mu'),         # Prior mean
    Real(0.01, 0.5, name='prior_sigma'),     # Prior sigma
    Integer(5, 200, name='epochs')         # Number of epochs
]


# Objective function to optimize using Gaussian Process
@use_named_args(space)
def objective_function(learning_rate, batch_size, layer1_units, layer2_units, prior_mu, prior_sigma, epochs, n=9):
    # Load the data for the specific dataset
    X = np.load(f'Datasets/kryptonite-{n}-X.npy')
    y = np.load(f'Datasets/kryptonite-{n}-y.npy')
    
    # Split the dataset into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
    X_val, _, y_val, _ = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Convert to torch tensors and create DataLoader instances
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)

    batch_size = int(max(16, batch_size))
    # DataLoader for training and validation
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

    # Initialize the model with the given hyperparameters
    model = BayesianNN(input_dim=X_train.shape[1], 
                       prior_mu=prior_mu, prior_sigma=prior_sigma, 
                       layer1_units=layer1_units, layer2_units=layer2_units)
    
    criterion = nn.BCELoss()  # Binary Cross-Entropy loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training the model for a fixed number of epochs
    for epoch in range(epochs):  # Use 10 epochs for each optimization trial
        train_bayesian_nn(model, train_loader, criterion, optimizer)
    
    # Evaluate the model on the validation set
    y_val_pred = evaluate_bayesian_nn(model, val_loader)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    
    # Return the negative validation accuracy to minimize (since gp_minimize tries to minimize the objective)
    return -val_accuracy



# Run Bayesian Optimization with Gaussian Process
results = gp_minimize(objective_function, space, n_calls=50, random_state=42, verbose=True)

# Print the best found hyperparameters
print("Best hyperparameters found:")
for param, value in zip(space, results.x):
    print(f"{param.name}: {value}")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 45.2703
Function value obtained: -0.4954
Current minimum: -0.4954
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 3.1798
Function value obtained: -0.4987
Current minimum: -0.4987
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 25.9858
Function value obtained: -0.5896
Current minimum: -0.5896
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 46.7780
Function value obtained: -0.9357
Current minimum: -0.9357
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 22.9944
Function value obtained: -0.5517
Current minimum: -0.9357
Iteration No: 6 start

In [1]:
# Define the search space for hyperparameters
thresholds = {
    '9': 0.95,
    '12': 0.925,
    '15': 0.90,
    '18': 0.875,
    '24': 0.80,
    '30': 0.75,
    '45': 0.70
}
# Objective function to optimize using Gaussian Process
@use_named_args(space)
def objective_function_all(learning_rate, batch_size, layer1_units, layer2_units, prior_mu, prior_sigma, epochs):
    total_loss = 0
    
    # Loop over different n values
    for n_str, threshold in thresholds.items():
        n = int(n_str)  # Convert n to integer
        
        # Load the data for the specific dataset
        X = np.load(f'Datasets/kryptonite-{n}-X.npy')
        y = np.load(f'Datasets/kryptonite-{n}-y.npy')
        
        # Split the dataset into training, validation, and test sets
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)
        X_val, _, y_val, _ = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
        
        # Convert to torch tensors and create DataLoader instances
        X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
        X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)
        
        batch_size = int(max(5, batch_size))
        # DataLoader for training and validation
        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)
        
        # Initialize the model with the given hyperparameters
        model = BayesianNN(input_dim=X_train.shape[1], 
                           prior_mu=prior_mu, prior_sigma=prior_sigma, 
                           layer1_units=layer1_units, layer2_units=layer2_units)
        
        criterion = nn.BCELoss()  # Binary Cross-Entropy loss
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        # Training the model for a fixed number of epochs
        for epoch in range(epochs):  # Use specified epochs for each optimization trial
            train_bayesian_nn(model, train_loader, criterion, optimizer)
        
        # Evaluate the model on the validation set
        y_val_pred = evaluate_bayesian_nn(model, val_loader)
        val_accuracy = accuracy_score(y_val, y_val_pred)
        
        # Calculate the loss for this n: the absolute deviation from the target threshold
        loss_n = abs(val_accuracy - threshold)
        total_loss += loss_n  # Add the loss for this n to the total loss
    
    # Return the total loss (to be minimized)
    return total_loss

# Run Bayesian Optimization with Gaussian Process
results = gp_minimize(objective_function_all, space, n_calls=50, random_state=42, verbose=True)

# Print the best found hyperparameters
print("Best hyperparameters found:")
for param, value in zip(space, results.x):
    print(f"{param.name}: {value}")

NameError: name 'use_named_args' is not defined

In [ ]:
# Run for each n value and collect accuracies
results = []
# learning_rate: 0.008705054471236994
# batch_size: 17
# layer1_units: 92
# layer2_units: 52
# prior_mu: 0.46796382181206203
# prior_sigma: 0.4384207640007753
# epochs: 100
# Acc: 0.9348

# learning_rate: 0.003992368422485689
# batch_size: 121
# layer1_units: 73
# layer2_units: 54
# prior_mu: 0.5103986684212475
# prior_sigma: 0.29950393862614066
# epochs: 81
# Accuracies across different n values: [(9, 0.9514814814814815), (12, 0.9186111111111112), (15, 0.866), (18, 0.505462962962963), (24, 0.5030555555555556), (30, 0.49977777777777777), (45, 0.49762962962962964)]

hyper_params9 = {
    'prior_mu': 0.5103986684212475,
    'prior_sigma': 0.29950393862614066,
    'layer1_units': 73,
    'layer2_units': 54,
    'batch_size': 121,
    'learning_rate': 0.003992368422485689,
    'epochs': 81
}
possible_n_vals = [9, 12, 15, 18, 24, 30, 45]
for n in tqdm(possible_n_vals):
    accuracy = run_bayesian_nn(n, hyper_params9)
    results.append((n, accuracy))

print("Accuracies across different n values:", results)

# Threshold grid for each n value
thresh_grid = {
    '9': 0.95,  
    '12': 0.925,
    '15': 0.90,
    '18': 0.875,
    '24': 0.80,
    '30': 0.75,
    '45': 0.70
}


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1, Train Loss: 0.7020
Epoch 2, Train Loss: 0.6968
Epoch 3, Train Loss: 0.6949
Epoch 4, Train Loss: 0.6948
Epoch 5, Train Loss: 0.6938
Epoch 6, Train Loss: 0.6945
Epoch 7, Train Loss: 0.6948
Epoch 8, Train Loss: 0.6929
Epoch 9, Train Loss: 0.6943
Epoch 10, Train Loss: 0.6927
Epoch 11, Train Loss: 0.6930
Epoch 12, Train Loss: 0.6929
Epoch 13, Train Loss: 0.6921
Epoch 14, Train Loss: 0.6906
Epoch 15, Train Loss: 0.6872
Epoch 16, Train Loss: 0.6793
Epoch 17, Train Loss: 0.6500
Epoch 18, Train Loss: 0.5991
Epoch 19, Train Loss: 0.5236
Epoch 20, Train Loss: 0.4401
Epoch 21, Train Loss: 0.3864
Epoch 22, Train Loss: 0.3364
Epoch 23, Train Loss: 0.3120
Epoch 24, Train Loss: 0.2797
Epoch 25, Train Loss: 0.2726
Epoch 26, Train Loss: 0.2469
Epoch 27, Train Loss: 0.2418
Epoch 28, Train Loss: 0.2408
Epoch 29, Train Loss: 0.2230
Epoch 30, Train Loss: 0.2213
Epoch 31, Train Loss: 0.2159
Epoch 32, Train Loss: 0.2139
Epoch 33, Train Loss: 0.2090
Epoch 34, Train Loss: 0.2091
Epoch 35, Train Loss: 0

 14%|█▍        | 1/7 [00:11<01:11, 11.88s/it]

Epoch 81, Train Loss: 0.1630
Validation Accuracy for n=9: 0.9481
Test Accuracy for n=9: 0.9515
Epoch 1, Train Loss: 0.7022
Epoch 2, Train Loss: 0.6956
Epoch 3, Train Loss: 0.6957
Epoch 4, Train Loss: 0.6940
Epoch 5, Train Loss: 0.6937
Epoch 6, Train Loss: 0.6939
Epoch 7, Train Loss: 0.6940
Epoch 8, Train Loss: 0.6940
Epoch 9, Train Loss: 0.6931
Epoch 10, Train Loss: 0.6927
Epoch 11, Train Loss: 0.6920
Epoch 12, Train Loss: 0.6935
Epoch 13, Train Loss: 0.6918
Epoch 14, Train Loss: 0.6912
Epoch 15, Train Loss: 0.6924
Epoch 16, Train Loss: 0.6924
Epoch 17, Train Loss: 0.6918
Epoch 18, Train Loss: 0.6910
Epoch 19, Train Loss: 0.6919
Epoch 20, Train Loss: 0.6911
Epoch 21, Train Loss: 0.6914
Epoch 22, Train Loss: 0.6910
Epoch 23, Train Loss: 0.6907
Epoch 24, Train Loss: 0.6911
Epoch 25, Train Loss: 0.6912
Epoch 26, Train Loss: 0.6900
Epoch 27, Train Loss: 0.6892
Epoch 28, Train Loss: 0.6891
Epoch 29, Train Loss: 0.6898
Epoch 30, Train Loss: 0.6894
Epoch 31, Train Loss: 0.6892
Epoch 32, Train

 29%|██▊       | 2/7 [00:27<01:10, 14.11s/it]

...

Test Accuracy for n=24: 0.5031
Epoch 1, Train Loss: 0.6989
Epoch 2, Train Loss: 0.6937
Epoch 3, Train Loss: 0.6944
Epoch 4, Train Loss: 0.6938
Epoch 5, Train Loss: 0.6938
Epoch 6, Train Loss: 0.6938
Epoch 7, Train Loss: 0.6937
Epoch 8, Train Loss: 0.6935
Epoch 9, Train Loss: 0.6934
Epoch 10, Train Loss: 0.6934
Epoch 11, Train Loss: 0.6934
Epoch 12, Train Loss: 0.6932
Epoch 13, Train Loss: 0.6936
Epoch 14, Train Loss: 0.6933
Epoch 15, Train Loss: 0.6932
Epoch 16, Train Loss: 0.6931
Epoch 17, Train Loss: 0.6933
Epoch 18, Train Loss: 0.6930
Epoch 19, Train Loss: 0.6930
Epoch 20, Train Loss: 0.6929
Epoch 21, Train Loss: 0.6925
Epoch 22, Train Loss: 0.6925
Epoch 23, Train Loss: 0.6925
Epoch 24, Train Loss: 0.6923
Epoch 25, Train Loss: 0.6923
Epoch 26, Train Loss: 0.6918
Epoch 27, Train Loss: 0.6915
Epoch 28, Train Loss: 0.6908
Epoch 29, Train Loss: 0.6905
Epoch 30, Train Loss: 0.6897
Epoch 31, Train Loss: 0.6896
Epoch 32, Train Loss: 0.6891
Epoch 33, Train Loss: 0.6875
Epoch 34, Train Loss:

 86%|████████▌ | 6/7 [02:27<00:30, 30.68s/it]

Test Accuracy for n=30: 0.4998
Epoch 1, Train Loss: 0.6969
Epoch 2, Train Loss: 0.6941
Epoch 3, Train Loss: 0.6935
Epoch 4, Train Loss: 0.6936
Epoch 5, Train Loss: 0.6935
Epoch 6, Train Loss: 0.6936
Epoch 7, Train Loss: 0.6934
Epoch 8, Train Loss: 0.6934
Epoch 9, Train Loss: 0.6933
Epoch 10, Train Loss: 0.6933
Epoch 11, Train Loss: 0.6933
Epoch 12, Train Loss: 0.6933
Epoch 13, Train Loss: 0.6932
Epoch 14, Train Loss: 0.6933
Epoch 15, Train Loss: 0.6934
Epoch 16, Train Loss: 0.6933
Epoch 17, Train Loss: 0.6932
Epoch 18, Train Loss: 0.6934
Epoch 19, Train Loss: 0.6932
Epoch 20, Train Loss: 0.6933
Epoch 21, Train Loss: 0.6932
Epoch 22, Train Loss: 0.6932
Epoch 23, Train Loss: 0.6933
Epoch 24, Train Loss: 0.6932
Epoch 25, Train Loss: 0.6932
Epoch 26, Train Loss: 0.6932
Epoch 27, Train Loss: 0.6933
Epoch 28, Train Loss: 0.6933
Epoch 29, Train Loss: 0.6932
Epoch 30, Train Loss: 0.6932
Epoch 31, Train Loss: 0.6932
Epoch 32, Train Loss: 0.6933
Epoch 33, Train Loss: 0.6932
Epoch 34, Train Loss:

100%|██████████| 7/7 [03:38<00:00, 31.28s/it]

Test Accuracy for n=45: 0.4976
Accuracies across different n values: [(9, 0.9514814814814815), (12, 0.9186111111111112), (15, 0.866), (18, 0.505462962962963), (24, 0.5030555555555556), (30, 0.49977777777777777), (45, 0.49762962962962964)]
